In [ ]:
# After importing the data, there are

# 8 measurements in the file
# The steps are, for each of the measurements:
# 1. Plot each measurement
# 2. Perform baseline subtractions
# 3. Determine the peaks
# 4. For each peak:
#    a. Find the the maximum value (beginning/end of peak?)
#    b. The integral of the peak
#    c. The time at which the peak reaches its maximum value.  (??)
# 5. Output the calculated data to a file, and plot it data.

In [ ]:
# import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tools
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d, splrep, splev
import plotly.express as px
import numpy as np
import pandas as pd
import scipy
import peakutils
from dataclasses import dataclass
from typing import List

In [ ]:
# @dataclass
# class Sample:
#     peak_start_idx: int | None
#     peak_end_idx: int | None
#     peak_max_idx: int | None
#     integrated_area: float | None
    
# @dataclass
# class Measurement:
#     id: str | None
#     samples: List[Sample] | None
        
# Measurement('i8/A', [])

 # Import the data file

In [ ]:



# Import data
df = pd.read_csv("Example_input.txt", header=25)
df.set_index('Time/s', inplace=True)
df.columns = df.columns.str.strip()  # Strip whitespace from the columns


# Create a dictionary for each column to store specific properties
measurements = {}
for col in df.columns:
    print(col)
    measurements[col] = {
        "baseline_spline_points": []  # Store selected baseline points
    }
    
    
# Add the subplots for each column
rows = 4
cols = 2
fig = go.FigureWidget(make_subplots(rows=rows, cols=cols))
# sample_count = input("How many samples are considered in the data?")
sample_count = 3  # For now, change this to an input

x = 0  # Column iterator. Assumes len(rows) + len(cols) == len(df.columns)
for row in [1, 2, 3, 4]:
    for col in [1, 2]:
        fig.add_trace(
            go.Scattergl(
                x=df.index, 
                y=df[df.columns[x]].values,
                name = df.columns[x],
                mode = 'lines+markers',
            ),
            row=row,
            col=col
        )
        
        chosen_points = sorted(measurements[df.columns[x]]['baseline_spline_points'])

        tck = splrep(df[df.columns[x]].index, df[df.columns[x]], t=df.loc[chosen_points].index, k=3)
        splinepoints = splev(df[df.columns[x]].index, tck)
        
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=splinepoints,
                mode='lines',
                name=f"spline_{df.columns[x]}",
                marker=dict(line=dict(color='black', width=2)),
            ),
            row=row,
            col=col
        )
        
        x=x+1  # Increment column assigned to 

        
DEFAULT_POINT_SIZE = 0
DEFAULT_POINT_COLOUR = '#a3a7e4'
        
clicked_points = []
# create our callback function
def update_point(trace, points, selector):
    # Must iterate through each plot in the subplot as everytime on_click is actuated, it returns a list
    for i, subplot in enumerate(fig.data):  
        if (
            i == points.trace_index 
            and len(points.point_inds)
#             and 'spline' not in points.trace_name
        ):  # Make sure we're working on the plot with changes
            point_color = list(subplot.marker.color)
            point_size = list(subplot.marker.size)
            
            for j in points.point_inds:
                # Point is given in a list, but we only click one at a time, unwrap it
                clicked_point = points.xs[0]
                # If a user clicks on the same point twice
                if clicked_point in measurements[points.trace_name]['baseline_spline_points']:
#                 if point_size[j] == 20:
                    print(f"Removing {clicked_point}")
                    point_color[j] = DEFAULT_POINT_COLOUR
                    point_size[j] = DEFAULT_POINT_SIZE
                    measurements[points.trace_name]['baseline_spline_points'].remove(clicked_point)
                else:
                    point_color[j] = '#2F4F4F'
                    point_size[j] = 20
                    measurements[points.trace_name]['baseline_spline_points'].append(clicked_point)
                
                with fig.batch_update():
                    subplot.marker.color = point_color
                    subplot.marker.size = point_size
                    print(f"x: {points.xs}, y:{points.ys} plot:{points.trace_index} ({subplot.name})")
                    
                    clicked_points.append((clicked_point, *points.ys))

                    

                    
                    for splineplot in fig.data:
                        if splineplot.name == f"spline_{points.trace_name}":
                            chosen_points = sorted(measurements[points.trace_name]['baseline_spline_points'])
                            tck = splrep(df[points.trace_name].index, df[points.trace_name], t=df.loc[chosen_points].index, k=3)
                            splinepoints = splev(df[points.trace_name].index, tck)
                            splineplot.y = splinepoints
                    print(measurements[points.trace_name]['baseline_spline_points'])

for subfig in fig.data:
    subfig.on_click(update_point)  # Callback for updating on click
    if 'spline' not in subfig.name:
        colors = [DEFAULT_POINT_COLOUR] * len(df)  
        subfig.marker.color = [DEFAULT_POINT_COLOUR] * len(df)  # Define the colour of points on first load
        subfig.marker.size = [DEFAULT_POINT_SIZE] * len(df)     # Same for size
        subfig.marker.line.color='DarkSlateGrey'
        subfig.marker.line.width=0

# Format and show fig
fig.update_layout(height=900, width=900, margin=dict(l=1, r=1, t=10, b=1))
fig

# Subtract the baseline

In [ ]:
pd.DataFrame(peakutils.baseline(df[' i1/A'], 3)).plot()

In [ ]:
pd.DataFrame()

# Select start:end of peak

# Integrate area in peak

# Find the peaks in the data

In [ ]:

from peakutils.plot import plot as pplot
from matplotlib import pyplot
%matplotlib inline

data = df[' i1/A']

peaks = peakutils.indexes(data, thres=0.5)

pyplot.figure(figsize=(10,6))
pplot(data.index, data, peaks)
pyplot.title('First estimate')
print(f"Peaks: {peaks}")